# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

### - This notebook is intended to be used with custom parallel data. That means that you need two files, where one is in your language, the other English, and the lines in the files are corresponding translations.

## Pre-process your data

We assume here that you already have a data set. The format in which we will process it here requires that 
1. you have two files, one for each language
2. the files are sentence-aligned, which means that each line should correspond to the same line in the other file.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ddn" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [0]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-ddn-baseline


In [0]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 2.7MB/s 


In [0]:
# TODO: specify the file paths here
source_file = "/test.en"
target_file = "/test.ddn"

# They should both have the same length.
! wc -l $source_file
! wc -l $target_file

7943 /my_file.en
7943 /my_file.ddn


In [0]:
# TODO: Pre-processing! (OPTIONAL)

# If your data contains weird symbols or the like, you might want to do some cleaning and normalization.
# We don't have the code in the notebook for that, but you can use sacremoses "normalize" for example for normalization punctuation: https://github.com/alvations/sacremoses.

# We apply tokenization to separate punctuation marks from the actual words, split words at hyphens etc.
# If your data is already tokenized, that's great! Skip this cell.
# Otherwise we can use sacremoses to do the tokenization for us. 
# We need the data to be tokenized such that it matches the global test set.

#! pip install sacremoses

#tok_source_file = source_file+".tok"
#tok_target_file = target_file+".tok"

# Tokenize the source
#! sacremoses tokenize -l $source_language < $source_file > $tok_source_file
# Tokenize the target
#! sacremoses tokenize -l $target_language < $target_file > $tok_target_file

# Let's take a look what tokenization did to the text.
#! head $source_file*
#! head $target_file*

# Change the pointers to our files such that we continue to work with the tokenized data.
#source_file = tok_source_file
#target_file = tok_target_file

In [0]:
# Download the global test set.
#! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
#os.environ["trg"] = target_language 
#os.environ["src"] = target_language 

#! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-$trg.en 
#! mv test.en-$trg.en test.en
#! wget https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
#! mv test.en-$trg.$trg test.$trg

# TODO: if this fails it means that there is NO test set for your language yet. It's on you to create one.
# A good idea would be to take a random subset of your data, and add it to https://raw.githubusercontent.com/masakhane-io/masakhane/master/jw300_utils/test/test.en-any.en.
# Make a Pull Request and get it approved and merged.
# Then repeat this cell to retrieve the new test set.
# Then proceed to the next cell that will filter out all duplicates from the training set, so that there is no overlap between training and test set.

In [0]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
#en_test_sents = set()
#filter_test_sents = "test.en-any.en"
#j = 0
#with open(filter_test_sents) as f:
#  for line in f:
#    en_test_sents.add(line.strip())
#    j += 1
#print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

In [0]:
import pandas as pd

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        #if line.strip() not in en_test_sents:
        source.append(line.strip())
        #else:
        #    skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        #if j not in skip_lines:
        target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 0/7942 lines since contained in test set.


,source_sentence,target_sentence
0,"The book of the generation of Jesus Christ, th...","Yesu Mɛsiyɑ ko ɑ̀ ci Dɑfidi ize, Abulɛmɑ ize, ..."
1,Abraham begat Isaac; and Isaac begat Jacob; an...,Abulɛmɑ nɑ Isɑɑkɑ hɛi. Isɑɑkɑ nɑ Yɑkɔfu hɛi. Y...
2,and Judah begat Perez and Zerah of Tamar; and ...,Yudɑ mo nɑ Fɑresi ndɑ Zerɑ hɛi Tɑmɑrɑ gɑɑ. Fɑr...


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [0]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
#df_pp.drop_duplicates(subset='source_sentence', inplace=True)
#df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

In [0]:
df_pp.shape

(7937, 2)

In [0]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv
import numpy as np

# TODO: if your corpus is smaller than 1000, reduce this number. With a corpus that small you might not obtain good results with NMT though :/
# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()


#Divide df_pp into train and test sets

msk = np.random.rand(len(df_pp)) < 0.98
train = df_pp[msk]
test_df = df_pp[~msk]

# Julia: test sets are already generated
dev_df = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped_df = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped_df.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev_df.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

with open("test."+source_language, "w") as src_file, open("test."+target_language, "w") as trg_file:
  for index, row in test_df.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")


#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)


# TODO: Doublecheck the format below. There should be no extra quotation marks or weird characters. It should also not be empty.
! head train.*
! head dev.*
! head test.*

==> train.ddn <==
À kɑɑ hunu kɑ hɑndunyɑ dimi yom kɑ ǹ goono hɑndunyɑ cɛnjɛ tɑɑci gɑɑ dɛrɑndi, wɑtom Gɔgu ndɑ Mɑgɔgu. À gɑ ǹ meigu wɑngu tɛyom sɛ. Ǹ bɑyom bisɑ tɛku tɑɑsi.
Zɑngɑ Ikpɛ Sendi hɑntumɑntɛ cii: Ikpɛ nɑ ǹ nɑ birikɑyom biyɑ. À nɑ ǹ nɑ moo yom kɑ ǹ si di, ndɑ hɑngɑ yom kɑ ǹ si mɑɑ hɑli kɑ kɑɑ hɔ.
A mɑɑ jinde beeri fɔ kɑ ɑ̀ go hunu zɑ Ikpɛ goonoyom dɔ. À go cii Ikpɛ dɔntɔneize iye di sɛ: Wɑ kpei kɑ Ikpɛ binefute gɑɑsiɑ iye munu hɑndunyɑ bɔm.
Kɑnkɑmi kunɑ ɑ nɑ gbei futu yom tɛ. A zɑm jiibi ce boobo. A mɑɑ hɛrɛɛ. A mɑɑ jo. A nɑ meehɔ yom zɑɑ cɛrɛ bɔm. A mɑɑ yeeni. A gɔrɔ bumbum.
Ammɑ Piɛɛ nɑ Yesu ze kɑ cii: A si ɑ̀ bei. A si fɑhɑm mo ndɑ hɛ kɑ n gɑ cii. Piɛɛ hunu hundi bɑtumɑ kunɑ kɑ kpei kɑtɑ dɔ. À gɑɑ no, gɔrɔngɔ cɑ jinde fɔ.
Ammɑ sɑɑ kɑ i nɑ jiribi iye tɔnɑndi, i tunu kɑ dirɑ. Ǹ kulu ndɑ ǹ wɛndɛ yom ndɑ ǹ koo yom nɑ i dum kɑlɑ wɑngɑrɑ bɑndɑ. I sɔmbu tɛkuɑ mee gɑɑ kɑ ɑduwɑ tɛ.
Yesu cii ɑ̀ sɛ: A koo hundiyo, n lɑɑkɑli mɑ si tunu. N nɑɑne nɑ n no bɑɑni. Kpei ndɑ lɑɑkɑli 

In [0]:
stripped_df.to_csv('train.csv')
!cp train.csv "/content/drive/My Drive/masakhane/$src-$tgt-$tag"

In [0]:
test_df.to_csv('test.csv')
!cp test.csv "/content/drive/My Drive/masakhane/$src-$tgt-$tag"

In [0]:
dev_df.to_csv('dev.csv')
!cp dev.csv "/content/drive/My Drive/masakhane/$src-$tgt-$tag"



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [0]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 2281 (delta 57), reused 46 (delta 25), pack-reused 2184
Receiving objects: 100% (2281/2281), 2.63 MiB | 2.83 MiB/s, done.
Resolving deltas: 100% (1578/1578), done.
Processing /content/joeynmt
     |████████████████████████████████| 276kB 6.1MB/s 
     |████████████████████████████████| 307kB 20.1MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 215kB 21.7MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 747kB 22.1MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=73054 sha256=c1198ffb12b088aaf489f5c473a170f06b899c645f2bc6228e4c8c4042b1b047
  Stored in directory: /tmp/pip-ephem-wheel-cache-uytgupvy/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyya

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [0]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Dendi Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.csv  test.bpe.ddn  test.ddn       train.bpe.en  train.en
dev.bpe.ddn	dev.ddn  test.bpe.en   test.en	      train.csv
dev.bpe.en	dev.en	 test.csv      train.bpe.ddn  train.ddn
bpe.codes.4000	dev.csv  test.bpe.ddn  test.ddn       train.bpe.en  train.en
dev.bpe.ddn	dev.ddn  test.bpe.en   test.en	      train.csv
dev.bpe.en	dev.en	 test.csv      train.bpe.ddn  train.ddn
BPE Dendi Sentences
Wom go hɑnu yom ndɑ hɑndu yom ndɑ I@@ ɔkɑci yom ndɑ jiiri yom lɑsɑbu kɑ bɛɛrɛ tɛ ǹ sɛ.
Bɑ ngɑ di bɑ si, i go fɑihã mo bɑ kɑnkɑmi yom kunɑ ko bei kɑ kɑnkɑmi wɔ ko@@ kɑri no ɑ̀ cini hɛi.
À nɑ ɑ dɑm gbei kunɑ bɑ kɑ ɑ ci Ikpɛ cɛn@@ ɑndikɔ ndɑ guruguz@@ ɑndikɔ ndɑ bɔrɔ fu@@ tu. Ammɑ ɑ du suuji domi zɑm kɑ bei kunɑ nɑ ɑ nɑ ɑ̀ tɛ zɑm kɑ nɑɑne kunɑ.
Jiribi nge yom di no coobɑɑbɑize yom go bɑ kɑ tɔntɔnu. Ammɑ Yuifu yom kɑ ǹ go Gɑrɛku cinɛ sendi go gun@@ u wɔ kɑ yom go E@@ bulu cinɛ sendi gɑɑ. Ǹ go gun@@ u no zɑmɑ E@@ bulu cinɛ send@@ ikɔ yom nɑ sɑɑle kɑɑ ǹ sokoni yom gɑɑ fɑrisime fɔ kunɑ kɑ n

In [0]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.csv  test.bpe.ddn  test.ddn       train.bpe.en  train.en
dev.bpe.ddn	dev.ddn  test.bpe.en   test.en	      train.csv
dev.bpe.en	dev.en	 test.csv      train.bpe.ddn  train.ddn


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 100                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [0]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-02-11 21:42:00,612 Hello! This is Joey-NMT.
2020-02-11 21:42:01,854 Total params: 12102656
2020-02-11 21:42:01,855 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [0]:
!ls joeynmt/models/${src}${tgt}_transformer

00004000.hyps.dev   2000.hyps  4000.hyps  8000.hyps	 tensorboard
00004000.hyps.test  3000.ckpt  5000.hyps  best.ckpt	 train.log
1000.hyps	    3000.hyps  6000.hyps  config.yaml	 trg_vocab.txt
2000.ckpt	    4000.ckpt  7000.hyps  src_vocab.txt  validations.txt


In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

cp: cannot create symbolic link '/content/drive/My Drive/masakhane/en-ddn-baseline/models/enddn_transformer/best.ckpt': Operation not supported


In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp joeynmt/models/${src}${tgt}_transformer/best.ckpt "$gdrive_path/models/${src}${tgt}_transformer/"

In [0]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 108471.50781	PPL: 33.64506	bleu: 2.13345	LR: 0.00030000	*
Steps: 2000	Loss: 93253.19531	PPL: 20.54460	bleu: 6.44712	LR: 0.00030000	*
Steps: 3000	Loss: 86804.96875	PPL: 16.66970	bleu: 9.49384	LR: 0.00030000	*
Steps: 4000	Loss: 85529.50781	PPL: 15.99461	bleu: 11.08313	LR: 0.00030000	*
Steps: 5000	Loss: 85867.19531	PPL: 16.17064	bleu: 11.61374	LR: 0.00030000	
Steps: 6000	Loss: 87552.13281	PPL: 17.07833	bleu: 12.36364	LR: 0.00030000	
Steps: 7000	Loss: 89329.66406	PPL: 18.09119	bleu: 13.38334	LR: 0.00030000	
Steps: 8000	Loss: 91286.44531	PPL: 19.27579	bleu: 14.04216	LR: 0.00030000	


In [0]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2020-02-11 22:15:09,857 Hello! This is Joey-NMT.
2020-02-11 22:15:32,580  dev bleu:  13.32 [Beam search decoding with beam size = 5 and alpha = 1.0]
Hello
2020-02-11 22:15:36,088 test bleu:  21.19 [Beam search decoding with beam size = 5 and alpha = 1.0]
